- 빅데이터 저장 및 처리기술
- 분산 저장 시스템: 많응 양 데이터 저장할 땐 분산 저장 필요. Hadoop이라는 분산 저장 시스템 사용해서 저장함. Aws S3등등...
- 이거 계산할 때도 분산 처리 프레임워크가 필요한데, 아파치 하둡, 등이 있음..
- 다루기 힘들기 땜에,,,못다룸

- 데이터 분석 초점 맞춰 수업
- 데이터 인코딩, 스케일링하는 경우가 대부분


## 고객의 구매 관련 데이터로 고객의 성별 분석

### 데이터 확인

In [1]:
# input
import pandas as pd
x_train=pd.read_csv('/Users/min0/Documents/Documents/school/4-1/빅데이터/자료/week13/x_train.csv', encoding='euckr')
x_test=pd.read_csv('/Users/min0/Documents/Documents/school/4-1/빅데이터/자료/week13/x_test.csv', encoding='euckr')
y_train=pd.read_csv('/Users/min0/Documents/Documents/school/4-1/빅데이터/자료/week13/y_train.csv', encoding='euckr')

In [3]:
x_train.head()
y_train.head()
x_test.head()

# x_train.describe()
# x_train.info()

# dir(x_train)
# help(x_train.shape)

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,3500,70900400,22000000,4050000.0,골프,부산본점,13,1.461538,0.789474,26
1,3501,310533100,38558000,48034700.0,농산물,잠실점,90,2.433333,0.369863,3
2,3502,305264140,14825000,30521000.0,가공식품,본 점,101,14.623762,0.083277,3
3,3503,7594080,5225000,NaN,주방용품,부산본점,5,2.000000,0.000000,47
4,3504,1795790,1411200,NaN,수산품,청량리점,3,2.666667,0.125000,8


### 데이터 전처리

In [6]:
# 테스트 데이터의 cust_id 저장
x_test_cust_id=x_test['cust_id']
# cust_id 컬럼 삭제
x_train.drop(columns=['cust_id'], inplace=True)
x_test.drop(columns=['cust_id'], inplace=True)
y_train.drop(columns=['cust_id'], inplace=True)

# 결측치 처리: 환불금액 널이면 0으로 처리
x_train.count()-x_train.notnull().count()
x_test.count()-x_test.notnull().count()
x_train['환불금액']=x_train['환불금액'].fillna(0)
x_test['환불금액']=x_test['환불금액'].fillna(0)

# 라벨 인코딩 수행하기
# 주 구매 상품과 지점을 레이블 인코딩: 골프, 농산물/잠실점 부산본점...
x_train['주구매상품'].unique().size
x_train['주구매지점'].unique().size
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
x_train['주구매상품']=encoder.fit_transform(x_train['주구매상품'])
x_train['주구매지점']=encoder.fit_transform(x_train['주구매지점'])
x_test['주구매상품']=encoder.fit_transform(x_test['주구매상품'])
x_test['주구매지점']=encoder.fit_transform(x_test['주구매지점'])
print(encoder.classes_)

# 파생변수 만들기
condition=x_train['환불금액'] > 0
x_train.loc[condition, '환불금액_new']=1
x_train.loc[~condition, '환불금액_new']=0
x_train.drop(columns=['환불금액'], inplace=True)
condition=x_test['환불금액'] > 0
x_test.loc[condition, '환불금액_new']=1
x_test.loc[~condition, '환불금액_new']=0
x_test.drop(columns=['환불금액'], inplace=True)

# 데이터 스케일링, 표준화 크기로 변환하기
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x=scaler.fit_transform(x_train)
x_train=pd.DataFrame(x, columns=x_train.columns)
x=scaler.transform(x_test)
x_test=pd.DataFrame(x, columns=x_test.columns)

# 불필요한 컬럼 삭제
x_train[['총구매액','최대구매액','환불금액_new']].corr()
x_train.drop(columns=['최대구매액'], inplace=True)
x_test.drop(columns=['최대구매액'], inplace=True)

['강남점' '관악점' '광주점' '노원점' '대구점' '대전점' '동래점' '미아점' '본  점' '부산본점' '부평점' '분당점'
 '상인점' '센텀시티점' '안양점' '영등포점' '울산점' '인천점' '일산점' '잠실점' '전주점' '창원점' '청량리점'
 '포항점']


In [7]:
# 분석
import sklearn.tree
dir(sklearn.tree)
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier(max_depth=10, criterion='entropy')
print(model.__doc__)
model.fit(x_train, y_train)
y_test_prob=model.predict_proba(x_test)
result=pd.DataFrame(y_test_prob)[1]
final=pd.concat([x_test_cust_id, result], axis=1).rename(columns={1:'gender'})
#final.to_csv('result.csv', index=False)

A decision tree classifier.

    Read more in the :ref:`User Guide <tree>`.

    Parameters
    ----------
    criterion : {"gini", "entropy"}, default="gini"
        The function to measure the quality of a split. Supported criteria are
        "gini" for the Gini impurity and "entropy" for the information gain.

    splitter : {"best", "random"}, default="best"
        The strategy used to choose the split at each node. Supported
        strategies are "best" to choose the best split and "random" to choose
        the best random split.

    max_depth : int, default=None
        The maximum depth of the tree. If None, then nodes are expanded until
        all leaves are pure or until all leaves contain less than
        min_samples_split samples.

    min_samples_split : int or float, default=2
        The minimum number of samples required to split an internal node:

        - If int, then consider `min_samples_split` as the minimum number.
        - If float, then `min_samples_spli

In [9]:
result

0       0.800000
1       0.000000
2       0.000000
3       0.451163
4       0.451163
          ...   
2477    0.462312
2478    0.451163
2479    0.560000
2480    0.000000
2481    0.512821
Name: 1, Length: 2482, dtype: float64

In [10]:
final

,cust_id,gender
0,3500,0.800000
1,3501,0.000000
2,3502,0.000000
3,3503,0.451163
4,3504,0.451163
...,...,...
2477,5977,0.462312
2478,5978,0.451163
2479,5979,0.560000
2480,5980,0.000000


In [8]:
# train-test split - 모델평가
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST=train_test_split(x_train, y_train, test_size=0.2, random_state=10)
print(X_TRAIN.shape)
import sklearn.tree
dir(sklearn.tree)
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier(max_depth=10, criterion='entropy')
print(model.__doc__)
model.fit(X_TRAIN, Y_TRAIN)
y_test_prob=model.predict_proba(X_TEST)
Y_TEST_PREDICTED=model.predict(X_TEST)
from sklearn.metrics import roc_auc_score
print(roc_auc_score(Y_TEST, Y_TEST_PREDICTED))

(2800, 8)
A decision tree classifier.

    Read more in the :ref:`User Guide <tree>`.

    Parameters
    ----------
    criterion : {"gini", "entropy"}, default="gini"
        The function to measure the quality of a split. Supported criteria are
        "gini" for the Gini impurity and "entropy" for the information gain.

    splitter : {"best", "random"}, default="best"
        The strategy used to choose the split at each node. Supported
        strategies are "best" to choose the best split and "random" to choose
        the best random split.

    max_depth : int, default=None
        The maximum depth of the tree. If None, then nodes are expanded until
        all leaves are pure or until all leaves contain less than
        min_samples_split samples.

    min_samples_split : int or float, default=2
        The minimum number of samples required to split an internal node:

        - If int, then consider `min_samples_split` as the minimum number.
        - If float, then `min_sa